In [2]:
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import MaxAbsScaler

In [5]:
df = utils.clean_df("../data/data.csv")


# Making a Machine learn model

As we've learned from our previous analisys, living area is gonna be the biggest predictor in our model. Let's take a look at it.

In [ ]:
plots.living_price(df[["Price", "Living_area"]])

Let's see and remove some outliers.

In [ ]:
df = utils.outliers(df)
plots.living_price(df[["Price", "Living_area"]])

Much better. A first attempt at modelling this data is doing a simple linear regression. Let's see how it goes:

In [ ]:
modelling.apply_model(LinearRegression(), df_fancy[["Price", "Living_area"]])[1]

Score is not great which is not exactly unexpected. We know living_area is a strong predictor of price, but we also know that things are more complex than that. Here we can use our intuition to gives us a direction.

What justifies the price difference between 2 homes with similar sizes on similar places?

I will define here something called "fancyness", and we will try to use our data to define that fancyness. If I had to guess, having an open fire sounds pretty fancy to me. Also having a large terrain (or surface of land) (for example over 2000 m²) might also be an indicator of fancyness. In our dataset we also have a "Subtype" of property, chosen by the person creating the listing. I believe the options
- Triplex
- Castle
- Exceptional Property

All indicate something about fancyness. So let's create our fancy data.

In [ ]:
def select_fancy_properties(df):
    fancy_subtypes = ['triplex', 'castle', "exceptional property"]
    df_fancy = df[df["Subtype"].isin(fancy_subtypes)]
    df_fancy = pd.concat([df_fancy
                        , df[df["Surface_of_land"] > 2000]
                        , df[df["Open Fire"] == True]
                        ])
    df_fancy.drop_duplicates(inplace=True)
    return df_fancy

df_fancy = select_fancy_properties(df)

In [ ]:
plots.living_price(df[["Price", "Living_area"]])

Seems to have a stronger correlation, let's apply a linear model to that subset of our data. 

In [ ]:
modelling.apply_model(LinearRegression(), df_fancy[["Price", "Living_area"]])[1]

It's better than before but so far we have used no information about location and we know the saying: real estate is all about location location location!

As we've learned in the analysis, the best location indicator we have is district, let's make our model based on that. We will only use the 10 districts with the most amount of listings so that we still get a reasonable dataset per district.

In [ ]:
district_list=df_fancy["District"].value_counts()[0:10].index.to_list()
result = m.apply_model_district(LinearRegression(), df_fancy, district_list)
print(result[1])

Better results, but what about the remaining lists that are not in our top 10 districts?

Let's take a look at how they behave together as a single unit.

In [ ]:
result = m.apply_model(LinearRegression(), df_fancy.loc[~df_fancy["District"].isin(district_list), ["Price", "Living_area"]])
print(result[1])

## Fancy residences summary

So far our model is performing ok for our so called fancy residences. Splitting our dataset seems like a good approach, both homes within our top 10 district as well as homes outside have a better relationship with living area than our entire dataset.

But if we are right about fancy residences behaving differently from the rest, by separating them we are not only making their predictions better but also helping the remaing dataset, since the fancy homes would have been outliers that made it harder to make a single linear regression, as we have done so far, work well.

So let's see if the remaining dataset is in a better position now.

In [ ]:
df_ordinary = df.loc[~df.index.isin(df_fancy["District"].index.to_list()), :]
plots.living_price(df_ordinary[["Price", "Living_area"]])

In [ ]:
result = m.apply_model(LinearRegression(), df_ordinary[["Price", "Living_area"]])
print(result[1])

It's not a particularly impressive result, but it's a better result than before which confirms our hypothesis that fancy houses behave differently indeed. But we also need to remember that this is the largest part of our original daatset and therefore should be the part that benefits the most from splitting by district. So let's do that, but now using 15 districts instead of 10 due to the larger amount of data points per district.

In [ ]:
district_list=df_ordinary["District"].value_counts()[0:15].index.to_list()
result = m.apply_model_district(LinearRegression(), df_ordinary, district_list)
print(result[1])

Another meaningful improvement, the only thing that's left is testing the remaining data, those outside our top 15 districts. This is gonna be the data that is left behind, the rest of the dataset that were not categorized, so we expect the worst result here.  